数据导入

In [ ]:
import pandas as pd
df=pd.read_csv(r"/content/drive/My Drive/KAGGLE/nlp/twitter/data/train.csv")
raw_x=df["text"].values.tolist()
raw_y=df["target"].values.tolist()
raw_data=list(zip(raw_x,raw_y))#x为推特文本 y为是否是灾难的标签

训练测试集划分 保存

In [ ]:
from sklearn.model_selection import train_test_split
training_data,test_data=train_test_split(raw_data,test_size=0.2, random_state=16)

In [ ]:
train=pd.DataFrame(training_data,columns=["text","target"])

In [ ]:
train.to_csv("/content/drive/My Drive/KAGGLE/nlp/twitter/train.csv")

模型及预处理部分

In [11]:
! pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 4.5MB/s 


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_pretrained_bert import BertModel
from torch.autograd import Variable
#下游任务模型
class bigru_attention(nn.Module):
    def __init__(self, bert_config, tagset_size, embedding_dim, hidden_dim, rnn_layers, dropout_ratio, dropout1, use_cuda):
        super(bigru_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.rnn_layers = rnn_layers
        self.word_embeds = BertModel.from_pretrained(bert_config)
        # 双向GRU，//操作为了与后面的Attention操作维度匹配，hidden_dim要取偶数！
        self.bigru = nn.GRU(embedding_dim, hidden_dim, num_layers=rnn_layers, bidirectional=True,dropout=dropout_ratio, batch_first=True)
        self.dropout1 = nn.Dropout(p=dropout1)
        # 由nn.Parameter定义的变量都为requires_grad=True状态
        self.weight_W = nn.Parameter(torch.Tensor(hidden_dim*2, hidden_dim*2))
        self.weight_proj = nn.Parameter(torch.Tensor(hidden_dim*2, 1))
        self.fc = nn.Linear(hidden_dim*2,tagset_size)
        nn.init.uniform_(self.weight_W, -0.1, 0.1)
        nn.init.uniform_(self.weight_proj, -0.1, 0.1)
        self.use_cuda =  use_cuda

    def rand_init_hidden(self, batch_size):
        if self.use_cuda:
            return Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)).cuda(), Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)).cuda()
        else:
            return Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)), Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim))
 
    def forward(self, sentence, attention_mask=None):
        batch_size = sentence.size(0)
        seq_length = sentence.size(1)
        embeds, _ = self.word_embeds(sentence, attention_mask=attention_mask, output_all_encoded_layers=False)
        hidden = self.rand_init_hidden(batch_size)
        gru_out, hiden = self.bigru(embeds) # [seq_len, bs, hid_dim]
        d_gru_out = self.dropout1(gru_out)
        x = d_gru_out
        # # # Attention过程，与上图中三个公式对应
        u = torch.tanh(torch.matmul(x, self.weight_W))
        att = torch.matmul(u, self.weight_proj)
        att_score = F.softmax(att, dim=1)
        scored_x = x * att_score
        # # # Attention过程结束
        feat = torch.sum(scored_x, dim=1)
        y = self.fc(feat)
        return y


备用

In [13]:
import pandas as pd
class InputFeatures(object):
    def __init__(self, text, label, input_id, input_mask):
        self.text = text
        self.label = label
        self.input_id = input_id
        self.input_mask = input_mask
#读取bert的词汇表 之后对单词进行编码
def load_vocab(vocab_file):
    vocab = {}
    index = 0
    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab
#读取训练 测试文件 分为输入texts 和标签labels
def load_file(file_path):
    df = pd.read_csv(file_path)
    raw_x=df["text"].values.tolist()
    raw_y=df["target"].values.tolist()
    raw_data=list(zip(raw_x,raw_y))
    texts = [[w for w in sample[0].split()] for sample in raw_data]
    labels = [sample[1] for sample in raw_data]
    return texts, labels
#进行对texts进行编码操作 补齐至最大长度
def load_data(file_path, max_length, vocab):
    texts, labels = load_file(file_path)
    assert len(texts) == len(labels)
    result = []
    for i in range(len(texts)):
        token = texts[i]
        label = int(labels[i])
        if len(token) > max_length-2:
            token = token[0:(max_length-2)]
        tokens_f =['[CLS]'] + token + ['[SEP]']
        input_ids = [int(vocab[i]) if i in vocab else int(vocab['[UNK]']) for i in tokens_f]
        mask_bool=1
        input_mask = [mask_bool] * len(input_ids)
        while len(input_ids) < max_length:
            input_ids.append(0)
            input_mask.append(0)
        assert len(input_ids) == max_length
        assert len(input_mask) == max_length
        #assert len(label_ids) == max_length 实体识别标签序列用
        feature = InputFeatures(text=tokens_f, label=label, input_id=input_ids, input_mask=input_mask)
        result.append(feature)
    return result

In [14]:
#文件位置以及超参数设置
save_model_dir='/content/drive/My Drive/KAGGLE/nlp/twitter/'
train_file='/content/drive/My Drive/KAGGLE/nlp/twitter/train.csv'
dev_file='/content/drive/My Drive/KAGGLE/nlp/twitter/test.csv'
max_length=30
vocab_file='/content/drive/My Drive/KAGGLE/nlp/twitter/bert/albert_base_v2.zip_files/vocab.txt'
batch_size=16
tagset_size = 2
epochs=20

模型训练及评估

In [15]:
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
if torch.cuda.is_available():
    device = torch.device("cuda", 0)
    print('device',device)
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False
vocab = load_vocab(vocab_file)
#load data
train_data = load_data(train_file, max_length=max_length, vocab=vocab)
train_ids = torch.LongTensor([temp.input_id for temp in train_data])
train_masks = torch.LongTensor([temp.input_mask for temp in train_data])
train_tags = torch.LongTensor([temp.label for temp in train_data])
train_dataset = TensorDataset(train_ids, train_masks, train_tags)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

dev_data = load_data(dev_file, max_length=max_length, vocab=vocab)
dev_ids = torch.LongTensor([temp.input_id for temp in dev_data])
dev_masks = torch.LongTensor([temp.input_mask for temp in dev_data])
dev_tags = torch.LongTensor([temp.label for temp in dev_data])
dev_dataset = TensorDataset(dev_ids, dev_masks, dev_tags)
dev_loader = DataLoader(dev_dataset, shuffle=True, batch_size=batch_size)

#模型及超参数设置        
model = bigru_attention('bert-base-cased', tagset_size, 768, 200, 1,
                      dropout_ratio=0.4, dropout1=0.4, use_cuda = use_cuda)
if use_cuda:
    model.cuda()
model.train()
losser=torch.nn.CrossEntropyLoss()
optimizer = getattr(optim, 'Adam')#优化器Adam
optimizer = optimizer(model.parameters(), lr=0.00003, weight_decay=0.00005)
best_f = -100
for epoch in range(epochs):
    print('epoch: {}trrain'.format(epoch))
    for i, train_batch in enumerate(tqdm(train_loader)):
        model.train()
        model.zero_grad()
        sentence, masks, tags = train_batch
        sentence, masks, tags = Variable(sentence), Variable(masks), Variable(tags)
        if use_cuda:
            sentence = sentence.cuda()
            masks = masks.cuda()
            tags = tags.cuda()
        loss = losser(model(sentence),tags)
        loss.backward()
        optimizer.step()
    print('epoch: {}loss: {}'.format(epoch, loss.item()))
    model.eval()
    pred = []
    true=[]
    for i, dev_batch in enumerate(dev_loader):#在验证集上验证
        model.zero_grad()
        sentence, masks, tags = dev_batch
        sentence, masks, tags = Variable(sentence), Variable(masks), Variable(tags)
        if use_cuda:
            sentence = sentence.cuda()
            asks = masks.cuda()
            tags = tags.cuda()
        predict_tags = F.softmax(model(sentence)).tolist()
        pred_tag=[i.index(max(i)) for i in predict_tags]
        pred.extend(pred_tag)
        true.extend(tags.tolist())
    print(classification_report(true, pred))
    f=f1_score(true, pred)
    if f > best_f:#保存最大f1模型
        model_name = save_model_dir + '.' + str(epochs) + ".pkl"
        torch.save(model.state_dict(), model_name)
        best_f = f

device cuda:0


  0%|          | 0/381 [00:00<?, ?it/s]

epoch: 0trrain


100%|██████████| 381/381 [00:53<00:00,  7.17it/s]


epoch: 0loss: 0.6927381753921509
              precision    recall  f1-score   support

           0       0.82      0.81      0.82       867
           1       0.76      0.76      0.76       656

    accuracy                           0.79      1523
   macro avg       0.79      0.79      0.79      1523
weighted avg       0.79      0.79      0.79      1523



...

epoch: 18loss: 0.08097044378519058


  0%|          | 1/381 [00:00<00:49,  7.69it/s]

              precision    recall  f1-score   support

           0       0.77      0.91      0.84       867
           1       0.84      0.64      0.73       656

    accuracy                           0.80      1523
   macro avg       0.81      0.78      0.78      1523
weighted avg       0.80      0.80      0.79      1523

epoch: 19trrain


100%|██████████| 381/381 [00:56<00:00,  6.71it/s]


epoch: 19loss: 0.0027462481521070004
              precision    recall  f1-score   support

           0       0.77      0.92      0.84       867
           1       0.86      0.64      0.73       656

    accuracy                           0.80      1523
   macro avg       0.82      0.78      0.79      1523
weighted avg       0.81      0.80      0.79      1523



预测

In [16]:
import torch
model =bigru_attention('bert-base-cased', tagset_size, 768, 200, 1,
                      dropout_ratio=0.4, dropout1=0.4, use_cuda = True)
model.load_state_dict(torch.load('/content/drive/My Drive/KAGGLE/nlp/twitter/.20.pkl'))#读取保存的模型参数
model.cuda()

bigru_attention(
  (word_embeds): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
          

预测测试集

In [21]:
test_file='/content/drive/My Drive/KAGGLE/nlp/twitter/data/test.csv'#读取测试集

In [25]:
class InputTestFeatures(object):
    def __init__(self, text, input_id, input_mask):
        self.text = text
        self.input_id = input_id
        self.input_mask = input_mask
def load_test_file(file_path):
    df = pd.read_csv(file_path)
    raw_x=df["text"].values.tolist()
    texts = [[w for w in sample.split()] for sample in raw_x]
    return texts
def load_test_data(file_path, max_length, vocab):
    texts = load_test_file(file_path)
    result = []
    for i in range(len(texts)):
        token = texts[i]
        if len(token) > max_length-2:
            token = token[0:(max_length-2)]
        tokens_f =['[CLS]'] + token + ['[SEP]']
        input_ids = [int(vocab[i]) if i in vocab else int(vocab['[UNK]']) for i in tokens_f]
        mask_bool=1
        input_mask = [mask_bool] * len(input_ids)
        while len(input_ids) < max_length:
            input_ids.append(0)
            input_mask.append(0)
        assert len(input_ids) == max_length
        assert len(input_mask) == max_length
        feature = InputTestFeatures(text=tokens_f, input_id=input_ids, input_mask=input_mask)
        result.append(feature)
    return result


In [39]:
#准备测试数据
test_data = load_test_data(test_file, max_length=max_length, vocab=vocab)
test_ids = torch.LongTensor([temp.input_id for temp in test_data])
test_masks = torch.LongTensor([temp.input_mask for temp in test_data])
test_dataset = TensorDataset(test_ids, test_masks)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

In [40]:
#测试
model.eval()
pred = []
for i, test_batch in enumerate(test_loader):
    model.zero_grad()
    sentence, masks= test_batch
    sentence, masks= Variable(sentence), Variable(masks)
    if use_cuda:
        sentence = sentence.cuda()
        asks = masks.cuda()
        tags = tags.cuda()
    predict_tags = F.softmax(model(sentence)).tolist()
    pred_tag=[i.index(max(i)) for i in predict_tags]
    pred.extend(pred_tag)

保存预测的测试数据

In [42]:
id=df["id"].values.tolist()

In [45]:
res_df=pd.DataFrame(zip(id,pred),columns=["id","target"])

In [48]:
res_df.to_csv("/content/drive/My Drive/KAGGLE/nlp/twitter/submission.csv",index=None)